In [1]:
import tensorflow as tf
import cv2
import numpy as np

In [2]:
# Downloaded .xml for face from cv2's github repo
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [3]:
dict = {0:'Angry', 1:'Disgust', 2:'Fear', 3:'Happy', 4:'Sad', 5:'Surprise', 6:'Neutral'}

#### Function for resizing the input buffer images

In [4]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):

    dim = None
    (h, w) = image.shape[:2]
    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        r = width / float(w)
        dim = (width, int(h * r))

    resized = cv2.resize(image, dim, interpolation = inter)
    # return the resized image
    return resized

In [5]:
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_SIMPLEX

with tf.Session() as sess:
    # change path according to location of model folder
    saver = tf.train.import_meta_graph(r'./model\sentiment-229.ckpt.meta')
    saver.restore(sess, tf.train.latest_checkpoint(r'./model/'))
    graph = tf.get_default_graph()
    tf_x = graph.get_tensor_by_name("tf_x:0")
    tf_keepprob = graph.get_tensor_by_name("keep_prob:0")
    y = graph.get_tensor_by_name("ArgMax:0")
    
    while 1:
        ret, img = cap.read()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x,y,w,h) in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
            IMG = img[y:y+h , x:x+w]
            gray = cv2.cvtColor(IMG,cv2.COLOR_BGR2GRAY)
            IMG = image_resize(gray,width = 48, height = 48)
            # rejecting not properly shaped images
            if IMG.shape[0] !=48 or IMG.shape[1] !=48:
                continue
            IMG = IMG.reshape(1,48,48,1)
            feed = {tf_x: IMG, tf_keepprob: 1.0}
            pred = sess.run(["ArgMax:0"],feed_dict=feed)
            cv2.putText(img, dict[pred[0][0]], (230, 50), font, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.imshow('img',img)
        # Press esc to exit 
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break

cap.release()
cv2.destroyAllWindows()

INFO:tensorflow:Restoring parameters from ./model/sentiment-229.ckpt
